# IMPORTS

Model imports

In [1]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

KFold cross validation imports

In [2]:
import splitfolders
from sklearn.model_selection import KFold, StratifiedKFold

c:\Users\maxba\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Device

In [3]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Image transformation

In [4]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

# Data loader

In [5]:
splitfolders.ratio("../Data", output="../DataTrainTest", seed=1337, ratio=(.8, 0, .2), group_prefix=None, move=False)

Copying files: 2465 files [00:09, 254.39 files/s]


Separation no KFold cross

# KFold cross validation

In [6]:
data_dir = '../Data'

In [ ]:
folds = StratifiedKFold(n_splits=10)

for train_index, test_index in folds.split(digits.data,digits.target):
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                       digits.target[train_index], digits.target[test_index]

In [9]:
train_path='../DataTrainTest/train/'
test_path='../DataTrainTest/test/'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [10]:
for i, (images,labels) in enumerate(train_loader):
    print(images)

tensor([[[[-0.9373, -0.9373, -0.9216,  ...,  0.0196,  0.0039,  0.0039],
          [-0.9137, -0.9216, -0.9137,  ...,  0.0196,  0.0275,  0.0118],
          [-0.9059, -0.9216, -0.9059,  ...,  0.0196,  0.0196, -0.0039],
          ...,
          [-0.9059, -0.8980, -0.9216,  ..., -0.8510, -0.8431, -0.8039],
          [-0.9137, -0.8980, -0.9216,  ..., -0.8824, -0.8275, -0.7961],
          [-0.9059, -0.9059, -0.9294,  ..., -0.8353, -0.7020, -0.7804]],

         [[-0.8824, -0.8824, -0.8667,  ...,  0.0667,  0.0510,  0.0510],
          [-0.8588, -0.8667, -0.8588,  ...,  0.0667,  0.0745,  0.0588],
          [-0.8510, -0.8667, -0.8431,  ...,  0.0667,  0.0667,  0.0431],
          ...,
          [-0.8588, -0.8588, -0.8824,  ..., -0.7725, -0.7647, -0.7333],
          [-0.8667, -0.8588, -0.8824,  ..., -0.7961, -0.7412, -0.7176],
          [-0.8588, -0.8667, -0.8902,  ..., -0.7490, -0.6078, -0.6941]],

         [[-0.9216, -0.9373, -0.9294,  ...,  0.2549,  0.2392,  0.2392],
          [-0.9059, -0.9294, -

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html

# Determine classes

In [11]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [12]:
print(classes)

['ClearSky', 'altocumulus', 'altostratus', 'cirrocumulus', 'cirrostratus', 'cirrus', 'cumulonimbus', 'cumulus', 'nimbostratus', 'stratocumulus', 'stratus']


# CNN Model

In [13]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=11):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            
        


In [14]:
model=ConvNet(num_classes=11).to(device)

# Optimizer

In [15]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [16]:
num_epochs=10

In [17]:
train_count=len(glob.glob(train_path+'/**/*.jpeg')) + len(glob.glob(train_path+'/**/*.jpg')) + len(glob.glob(train_path+'/**/*.bmp')) + len(glob.glob(train_path+'/**/*.png'))
test_count=len(glob.glob(test_path+'/**/*.jpeg')) + len(glob.glob(test_path+'/**/*.jpg')) + len(glob.glob(test_path+'/**/*.bmp')) + len(glob.glob(test_path+'/**/*.png'))

In [18]:
print(train_count,test_count)

1968 496


# Training

In [19]:
best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(28.7737) Train Accuracy: 0.179369918699187 Test Accuracy: 0.18951612903225806
Epoch: 1 Train Loss: tensor(9.4403) Train Accuracy: 0.33384146341463417 Test Accuracy: 0.2721774193548387
Epoch: 2 Train Loss: tensor(4.9484) Train Accuracy: 0.4883130081300813 Test Accuracy: 0.36088709677419356
Epoch: 3 Train Loss: tensor(3.1755) Train Accuracy: 0.5904471544715447 Test Accuracy: 0.35080645161290325
Epoch: 4 Train Loss: tensor(2.4487) Train Accuracy: 0.6590447154471545 Test Accuracy: 0.3004032258064516
Epoch: 5 Train Loss: tensor(1.9299) Train Accuracy: 0.7347560975609756 Test Accuracy: 0.36693548387096775
Epoch: 6 Train Loss: tensor(1.6059) Train Accuracy: 0.7505081300813008 Test Accuracy: 0.3387096774193548
Epoch: 7 Train Loss: tensor(1.8786) Train Accuracy: 0.7362804878048781 Test Accuracy: 0.3548387096774194
Epoch: 8 Train Loss: tensor(1.3155) Train Accuracy: 0.8119918699186992 Test Accuracy: 0.3971774193548387
Epoch: 9 Train Loss: tensor(1.4581) Train Accuracy